# 店舗idに関しては一度行えば良いものなので関数化をしなかった。

In [146]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 例外処理用のlibraryをimport
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
#数字とアルファベットのみ半角に変換したいため使用
import unicodedata

In [18]:
driver = webdriver.Chrome()
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="b870bff1fbc8ae81918922cc6e982465")>>

下記ページのすべての店舗とその他のページをスクレイピングする

In [20]:
url='https://www.8190.jp/shop/'
driver.get(url)
time.sleep(2)

# 各店舗のboxをリストで取得して、for文で一店舗ごと取得していく

## テスト

In [134]:
shop_info =driver.find_elements(By.CLASS_NAME,'textBox')
tenpo=shop_info[4].find_elements(By.CLASS_NAME,'txt')
address = shop_info[4].find_elements(By.CLASS_NAME,'address')
phone = shop_info[4].find_elements(By.CLASS_NAME,'wid')
phone_2 = shop_info[4].find_elements(By.XPATH,'./ul/li')

In [115]:
len(shop_info)

81

In [118]:
tenpo[0].text

'郡山店'

In [103]:
len(tenpo)

1

In [137]:
phone_2[1].text

'販売相談\n0120-37-8190'

In [123]:
if len(phone) ==0:
    phone = shop_info[4].find_elements(By.XPATH,'./ul/li')

0

In [105]:
tenpo[0].text

'札幌店'

In [86]:
address[80].text

'〒900-0004\n沖縄県那覇市銘苅266-3'

In [101]:
phone[4].text

'お問い合わせ\n029-843-8190'

In [24]:
wrapper.text

'北海道・東北エリア\n北海道\n札幌店買取\n〒007-0861\n北海道札幌市東区伏古一条4-1-1\n営業時間・定休日\n10:00～19:00(平日)\u300010:00～19:00(土日祝)\n定休日\u3000木曜日\nご来店について\u3000無料出張買取のご依頼状況により、店舗にスタッフが不在となる場合がございます。\nご来店いただく際には、下記連絡先までご連絡いただき、ご確認いただくと確実な対応が可能ですので、お気軽にお問い合わせください。\nお問い合わせ\n0120-65-8190\nもっと見る\n岩手県\n盛岡店販売買取\n〒020-0836\n岩手県盛岡市津志田西2-17-50\u3000盛岡南ショッピングセンターサンサ内\n営業時間・定休日\n10:00～19:00(平日)\u300010:00～19:00(土日祝)\n定休日\u3000木曜日\nお問い合わせ\n019-601-8190\nもっと見る\n宮城県\n仙台店販売レンタル買取\n〒981-3117\n宮城県仙台市泉区市名坂字黒木川原54-1\n営業時間・定休日\n10:00～19:00(平日)\u300010:00～19:00(土日祝)\n定休日\u3000火曜日\nお問い合わせ\n022-772-8190\nもっと見る\n福島県\nラパークいわき店販売レンタル買取\n〒970-8044\n福島県いわき市中央台飯野4-1（ラパークいわき内）\n営業時間・定休日\n10:00～19:00(平日)\u300010:00～19:00(土日祝)\n定休日\u3000月曜（祝祭日を除く）\nご来店について\u3000みなさまのご来店心よりお待ちしております。\nお問い合わせ\n0246-38-9980\nもっと見る\n郡山店販売買取\n〒963-8022\n福島県郡山市西ノ内2-20-7\n営業時間・定休日\n10:00～19:00(平日)\u300010:00～19:00(土日祝)\n定休日\u3000水曜日\n買取相談\n0120-65-8190\n販売相談\n0120-37-8190\n※「買取」「販売」で電話番号が異なります。お掛け間違いにご注意ください。\nもっと見る'

In [43]:
tenpo=driver.find_elements(By.CLASS_NAME,'txt')

## 実行


In [138]:
not phone

True

In [144]:
columns =['店舗名','郵便番号','住所','電話番号']
df = pd.DataFrame(index=[],columns=columns)
shop_info =driver.find_elements(By.CLASS_NAME,'textBox')
for shop in shop_info:
    tenpo=shop.find_elements(By.CLASS_NAME,'txt')[0].text
    post_code=  shop.find_elements(By.CLASS_NAME,'address')[0].text.split('\n')[0]
    address = shop.find_elements(By.CLASS_NAME,'address')[0].text.split('\n')[1]
    phone_element = shop.find_elements(By.CLASS_NAME,'wid')
    if not phone_element:
        phone_element = shop.find_elements(By.XPATH,'./ul/li')
    phone = phone_element[0].text.split('\n')[1]
    #pandasSeriesの作成
    transaction =[tenpo,post_code,address,phone]
    record = pd.Series(transaction,index=df.columns)
    df.loc[len(df)] = record

In [145]:
df

,店舗名,郵便番号,住所,電話番号
0,札幌店,〒007-0861,北海道札幌市東区伏古一条4-1-1,0120-65-8190
1,盛岡店,〒020-0836,岩手県盛岡市津志田西2-17-50 盛岡南ショッピングセンターサンサ内,019-601-8190
2,仙台店,〒981-3117,宮城県仙台市泉区市名坂字黒木川原54-1,022-772-8190
3,ラパークいわき店,〒970-8044,福島県いわき市中央台飯野4-1（ラパークいわき内）,0246-38-9980
4,郡山店,〒963-8022,福島県郡山市西ノ内2-20-7,0120-65-8190
...,...,...,...,...
76,小倉店,〒802-0971,福岡県北九州市小倉南区守恒本町２丁目１−３３,0120-65-8190
77,Beat&C 福岡マリナ店,〒819-0001,福岡県福岡市西区小戸３丁目５１−４７,0120-65-8190
78,熊本店,〒862-0963,熊本県熊本市南区出仲間7-15-15,0120-65-8190
79,上熊本店,〒860-0079,熊本県熊本市西区上熊本2-17−39,0120-65-8190


In [147]:
#数字を半角に変換
def zen_to_han_number(text):
    text = unicodedata.normalize('NFKC',text)
    return text


In [148]:
df['郵便番号']=df['郵便番号'].apply(zen_to_han_number)
df['住所'] = df['住所'].apply(zen_to_han_number)
df['電話番号']=df['電話番号'].apply(zen_to_han_number)

In [149]:
df

,店舗名,郵便番号,住所,電話番号
0,札幌店,〒007-0861,北海道札幌市東区伏古一条4-1-1,0120-65-8190
1,盛岡店,〒020-0836,岩手県盛岡市津志田西2-17-50 盛岡南ショッピングセンターサンサ内,019-601-8190
2,仙台店,〒981-3117,宮城県仙台市泉区市名坂字黒木川原54-1,022-772-8190
3,ラパークいわき店,〒970-8044,福島県いわき市中央台飯野4-1(ラパークいわき内),0246-38-9980
4,郡山店,〒963-8022,福島県郡山市西ノ内2-20-7,0120-65-8190
...,...,...,...,...
76,小倉店,〒802-0971,福岡県北九州市小倉南区守恒本町2丁目1−33,0120-65-8190
77,Beat&C 福岡マリナ店,〒819-0001,福岡県福岡市西区小戸3丁目51−47,0120-65-8190
78,熊本店,〒862-0963,熊本県熊本市南区出仲間7-15-15,0120-65-8190
79,上熊本店,〒860-0079,熊本県熊本市西区上熊本2-17−39,0120-65-8190


In [153]:
driver = webdriver.Chrome()
driver.close
url='https://www.8190.jp/shop/'
driver.get(url)
time.sleep(2)

In [171]:
#関数化(pageclickがめんどくさい+店舗数は大きく変わるものではないので、driverで操作してから取得する)
def tenpo_scrape(driver):
    columns =['店舗名','郵便番号','住所','電話番号']
    df = pd.DataFrame(index=[],columns=columns)
    shop_info =driver.find_elements(By.CLASS_NAME,'textBox')
    for shop in shop_info:
        tenpo=shop.find_elements(By.CLASS_NAME,'txt')[0].text
        post_code=  shop.find_elements(By.CLASS_NAME,'address')[0].text.split('\n')[0]
        address = shop.find_elements(By.CLASS_NAME,'address')[0].text.split('\n')[1]
        phone_element = shop.find_elements(By.CLASS_NAME,'wid')
        if not phone_element:
            phone_element = shop.find_elements(By.XPATH,'./ul/li')
        phone = phone_element[0].text.split('\n')[1]
        #pandasSeriesの作成
        transaction =[tenpo,post_code,address,phone]
        record = pd.Series(transaction,index=df.columns)
        df.loc[len(df)] = record
        df['郵便番号']=df['郵便番号'].apply(zen_to_han_number)
        df['住所'] = df['住所'].apply(zen_to_han_number)
        df['電話番号']=df['電話番号'].apply(zen_to_han_number)
        driver.close
    return df


In [172]:
df_other = tenpo_scrape(driver)

In [173]:
df_other

,店舗名,郵便番号,住所,電話番号
0,Husqvarna Motorcycles いわき店,〒970-8044,福島県いわき市中央台飯野4-1(ラパークいわき内),0246-38-9659
1,KTM小山,〒323-0824,栃木県小山市雨ケ谷新田69-1,0120-65-8190


In [174]:
df_join = pd.concat([df,df_other]).reset_index()

In [175]:
df_join.drop(columns='index',inplace=True)

df_joinのindexを店舗idとして使用する

In [176]:
df_join['店舗id'] = df_join.index

In [177]:
df_join

,店舗名,郵便番号,住所,電話番号,店舗id
0,札幌店,〒007-0861,北海道札幌市東区伏古一条4-1-1,0120-65-8190,0
1,盛岡店,〒020-0836,岩手県盛岡市津志田西2-17-50 盛岡南ショッピングセンターサンサ内,019-601-8190,1
2,仙台店,〒981-3117,宮城県仙台市泉区市名坂字黒木川原54-1,022-772-8190,2
3,ラパークいわき店,〒970-8044,福島県いわき市中央台飯野4-1(ラパークいわき内),0246-38-9980,3
4,郡山店,〒963-8022,福島県郡山市西ノ内2-20-7,0120-65-8190,4
...,...,...,...,...,...
78,熊本店,〒862-0963,熊本県熊本市南区出仲間7-15-15,0120-65-8190,78
79,上熊本店,〒860-0079,熊本県熊本市西区上熊本2-17−39,0120-65-8190,79
80,那覇店,〒900-0004,沖縄県那覇市銘苅266-3,0120-65-8190,80
81,Husqvarna Motorcycles いわき店,〒970-8044,福島県いわき市中央台飯野4-1(ラパークいわき内),0246-38-9659,81


In [178]:
# df_join.to_csv('../data/tenpo_data.csv')